In [14]:
import sqlalchemy

### Exercise 2

* List the users who registered in 2018 with a `.com` email address and living in China

```mysql
SELECT * 
FROM users   
WHERE email LIKE '%.com' AND  
    EXTRACT(YEAR FROM date_of_registration) = 2018 AND  
    country = 'China';
```

* How many users are there?

```mysql
SELECT COUNT(*) 
FROM users;
```

* How many users registered in 2019?
```mysql
SELECT COUNT(*) 
FROM users
WHERE EXTRACT(YEAR FROM date_of_registration) = 2019;
```

* How many users registered in January?
```mysql
SELECT * 
FROM users
WHERE EXTRACT(MON FROM date_of_registration) = 1;
```

* Which country has the most users?
```mysql
SELECT country, COUNT(*) AS N_users 
FROM users
GROUP BY country
ORDER BY N_users DESC
LIMIT 1;
```

* What is the gender ratio?

Solution 1.
```mysql
SELECT
    SUM ( CASE WHEN gender = 'Male' THEN 1 ELSE 0 END )::DECIMAL 
    /
    SUM ( CASE WHEN gender = 'Female' THEN 1 ELSE 0 END ) 
FROM users;
```

Solution 2.
```mysql
SELECT (
    SELECT COUNT(*)
    FROM users
    WHERE gender = 'Male'
    )::DECIMAL
    /
    (
     SELECT COUNT(*)
     FROM users
     WHERE gender = 'Female'
    );
```

* How many users left at least one field blank?
```mysql
SELECT COUNT(*) 
FROM users
WHERE gender IS NULL
    OR country IS NULL
    OR last_name IS NULL
    OR first_name IS NULL
    OR date_of_registration IS NULL
    OR email IS NULL
    OR username IS NULL;
```

* Which are the 3 most expensive products?
```mysql
SELECT name
FROM products
ORDER BY price DESC
LIMIT 3;
```

* Which are the 4th and 5th cheapest products?
```mysql
SELECT name
FROM products
ORDER BY price DESC
LIMIT 2 OFFSET 3;
```

* What is the average price for an electric product?
```mysql
SELECT ROUND(AVG(price), 2) AS ave_price
FROM products
WHERE category = 'Electronics';
```

* How much would it cost me to buy all the toys?
```mysql
SELECT SUM(price) AS cost
FROM products
WHERE category = 'Toys';
```

* What is the average rating?
```mysql
SELECT AVG(rating)
FROM reviews;
```

* Which product has the best average rating?
```mysql
/* THIS ONE IS WRONG! */
SELECT products.name, AVG(reviews.rating) AS avg_rating
FROM products
INNER JOIN reviews
    ON products.id = reviews.product_id
GROUP BY products.name /* Should group by products.id */
ORDER BY avg_rating DESC
LIMIT 10;
```
```mysql
/* This suppose to be the correct one */
SELECT products.name, avg_rating
FROM products
INNER JOIN (SELECT product_id, AVG(reviews.rating) AS avg_rating
            FROM reviews
            GROUP BY product_id
           ) AS avg_review
           ON avg_review.product_id = products.id
ORDER BY avg_rating DESC 
LIMIT 10;
``` 

* Which product has the worst rating?
```mysql
SELECT products.name, avg_rating
FROM products
INNER JOIN (SELECT product_id, AVG(reviews.rating) AS avg_rating
            FROM reviews
            GROUP BY product_id
           ) AS avg_review
           ON avg_review.product_id = products.id
ORDER BY avg_rating
LIMIT 10;
``` 

* Which products have no review?
```mysql
SELECT *
FROM products
LEFT JOIN reviews
    ON products.id = reviews.product_id
WHERE reviews.product_id IS NULL;
```

* How many reviews are 3 or below without comment?
```mysql
SELECT COUNT(*)
FROM reviews
WHERE reviews.comment IS NULL AND
    rating < 3;
```

* Which user reviewed the most?
```mysql
SELECT users.username
FROM users
WHERE users.id = (
    SELECT users.id
    FROM users
    INNER JOIN reviews
        ON reviews.user_id = users.id
    GROUP BY users.id
    ORDER BY COUNT(*) DESC
    LIMIT 1
    );
```

* List the average rating for each product

```mysql
SELECT products.name, avg_rating
FROM products
INNER JOIN(
    SELECT product_id, AVG(reviews.rating) AS avg_rating
    FROM reviews
    GROUP BY product_id
    ) AS avg_review
    ON avg_review.product_id = products.id;
```

* How many days passed since the last review?
```mysql
SELECT EXTRACT(DAY FROM NOW() - (SELECT reviews.date 
                                 FROM reviews 
                                 ORDER BY reviews.date DESC
                                 LIMIT 1));
```

### Exercise 3

We would like to store data about our employees.

- First name
- Last name
- Username
- Date of employment
- Date of exit
- Role
- Salary

Your task is to

- Create the necessary tables and maybe add the necessary fields
- Alter the existing tables so that an employee belongs to a department as well
- Remove the username field from the table
- Rename a column

```mysql
CREATE TABLE employees(
    id SERIAL PRIMARY KEY,
    first_name VARCHAR,
    last_name VARCHAR,
    username VARCHAR,
    date_of_employment TIMESTAMP,
    date_of_exit TIMESTAMP,
    role VARCHAR,
    Salary_in_cents INTEGER);
```

```mysql
ALTER TABLE employees
ADD COLUMN department VARCHAR;
```

```mysql
ALTER TABLE employees
DROP COLUMN username;
```

```mysql
ALTER TABLE employees
RENAME COLUMN Salary_in_cents TO Salary;
```